In [14]:
# pip install tavily-python

import os
import json
import time
from openai import OpenAI
from tavily import TavilyClient
from IPython.display import display, Markdown

# Initialize clients with API keys
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

prompt = """You are an immigration expert having expertise in Finnish immigration and integration. 
Your goal is to provide updated information from the internet using Tavily search API function. 
If no relevant information is found on the internet, utilize your own knowledge
to answer the question. In either case, acknowledge the source of the information. 
If the information is not present in your own knowledge, simply say that you cannot provide the requested information.
In case of an internet search, Please include relevant url sources at the end of your answers with the heading "Citations".
Analyze the query to determine whether the question requires internet access to find the updated information. 
Internet access may be required for queries asking for information on rapidly changing policies, current statistics, or other recent facts (e.g., current tax rate, tution fees, visa processing fee, residency period, etc.
It is important to note that you only have to answer queries related to Finland in general (e.g, location, history, economy, etc.), Finnish immigration,
Finnish culture, Finnish lifestyle, or integration in Finland. For all other non-related queries, simply respond by giving an appropriate message
"""

# Create an assistant
assistant = client.beta.assistants.create(
    instructions=prompt,
    model="gpt-4o-mini",
    tools=[{
        "type": "function",
        "function": {
            "name": "tavily_search",
            "description": "Get information on recent events from the web.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "The search query to use. For example: 'What is the average processing time of residence permit?'"},
                },
                "required": ["query"]
            }
        }
    }]
)
assistant_id = assistant.id
print(f"Assistant Created. Assistant ID: {assistant_id}")

Assistant Created. Assistant ID: asst_TVF43xGGu6SqHjO5EP54gKnY


In [15]:
"""Create a thread for the assistant and maintain chat."""
import os
import json
import time
from openai import OpenAI
from tavily import TavilyClient
from IPython.display import display, Markdown

# Function to perform a Tavily search
def tavily_search(query):
    display(Markdown("Retrieving updated information..."))  # Status message when using TavilyClient
    search_result = tavily_client.get_search_context(query, search_depth="advanced", max_tokens=8000)
    return search_result

# Function to wait for a run to complete
def wait_for_run_completion(thread_id, run_id):
    while True:
        display(Markdown("Thinking..."))
        time.sleep(1)
        run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
        if run.status in ['completed', 'failed', 'requires_action']:
            return run

# Function to handle tool output submission
def submit_tool_outputs(thread_id, run_id, tools_to_call):
    tool_output_array = []
    for tool in tools_to_call:
        output = None
        tool_call_id = tool.id
        function_name = tool.function.name
        function_args = tool.function.arguments

        if function_name == "tavily_search":
            output = tavily_search(query=json.loads(function_args)["query"])

        if output:
            tool_output_array.append({"tool_call_id": tool_call_id, "output": output})

    return client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread_id,
        run_id=run_id,
        tool_outputs=tool_output_array
    )

# Function to print messages from a thread, ensuring they appear in the correct order
def print_messages_from_thread(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    sorted_messages = sorted(messages, key=lambda msg: msg.created_at)  # Sort by creation time
    for msg in sorted_messages:
        display(Markdown(f"{msg.role}: {msg.content[0].text.value}"))

# Create a thread
thread = client.beta.threads.create()
print(f"Thread: {thread}")


# Ongoing conversation loop
while True:
    display(Markdown("**You (type 'exit' to end):**"))
    user_input = input()
    if user_input.lower() == 'exit':
        break

    # Create a message
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=user_input,
    )

    # Create a run
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )

    # Wait for run to complete
    run = wait_for_run_completion(thread.id, run.id)

    if run.status == 'failed':
        print(run.error)
        continue
    elif run.status == 'requires_action':
        run = submit_tool_outputs(thread.id, run.id, run.required_action.submit_tool_outputs.tool_calls)
        run = wait_for_run_completion(thread.id, run.id)

    # Print messages from the thread
    print_messages_from_thread(thread.id)


Thread: Thread(id='thread_fsRlmIs6o8CTe1phrOvaWuS6', created_at=1727951260, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


**You (type 'exit' to end):**

  What is the current residency period for citizenship?


Thinking...

Retrieving updated information...

...

assistant: Currently, the period of residence required for acquiring Finnish citizenship is five years. However, this period will be extended to eight years starting from October 1, 2024. The new law will require applicants to have lived in Finland for a continuous period of eight years, emphasizing successful integration as a prerequisite for citizenship.

If an application is submitted before October 1, 2024, the five-year requirement will still apply.

Citations:
- [Valtioneuvosto](https://migri.fi/en/-/longer-period-of-residence-required-for-citizenship-as-of-1-october-2024)
- [Valtioneuvosto](https://valtioneuvosto.fi/en/-/1410869/stricter-requirements-for-acquiring-finnish-citizenship-the-required-period-of-residence-to-be-extended)
- [Migri](https://migri.fi/en/period-of-residence)

user: What will be the change in the tuition fees from 2024?

assistant: Starting in 2024, there will be significant changes to tuition fees in Finnish universities for non-EU and non-EEA students. The proposed legislative changes mean that these students will be required to pay full tuition fees that cover the total cost of their education, along with an application fee.

For specific institutions, for example:
- At Tampere University of Applied Sciences (TAMK), the tuition fee for non-EU and non-EEA students will rise to €12,000 per academic year. This is an increase from the previous fees of €9,800 for Bachelor's degrees and €10,800 for Master's degrees for students who started their studies in 2023 or earlier.

These changes are aimed at improving the financial sustainability of higher education institutions and encouraging international students to remain in Finland after graduation.

Citations:
- [Yle](https://yle.fi/a/74-20089083)
- [TAMK](https://www.tuni.fi/en/students-guide/handbook/tamk/starting-your-studies/studying-tampere/tuition-fees)

**You (type 'exit' to end):**

 exit
